# В зависимости от уровня объема торгов (квартал, 2 года, весь период)

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import time

def p_v_kv(file): #функция для вывода p-value срзнач логдоходностей по кварталам для одной компании
    df1 = pd.read_csv(file, sep=';', encoding='cp1251') #считываем csv файл
    kv = [(1, 3), (4, 6), (7, 9), (10, 12)] #список месяцев начала и конца кварталов
    df_mean = [] #среднее значение лог доходностей за каждый квартал
    for year in range(2011, 2019): #так как 2010 и 2020 года взяты не полностью
        date = year*100 #формируем дату для условия одного квартала
        for i in kv:
            condition = (df1['<DATE>'] >= ((date + i[0])*100 + 1)) & (df1['<DATE>'] <= ((date + i[1])*100 + 31)) #условие для одного квартала
            df = pd.DataFrame()
            df['close'] = df1[condition]['<CLOSE>'] #отбираются цены закрытия для данного периода
            df['open'] = df1[condition]['<OPEN>'] #отбираются цены открытия для данного периода
            df['vol'] = df1[condition]['<VOL>'] #отбираются объем торгов в шт для данного периода
            df['rvol'] = ((df['close'] + df['open']) / 2) * df['vol']
            df = pd.DataFrame.diff(np.log(df['rvol'] ))[1::] #подсчет логдохоности
            df_mean.append(df.mean()) #добавление срднего значения за квартал
    return  round(st.shapiro(df_mean)[1], 5) #p-value по средним квартальным логдоходностям для одной компании
                                                                
def p_v_2_years(file): #функция для вывода p-value срзнач логдоходностей для двух лет для одной компании
    df1 = pd.read_csv(file, sep=';', encoding='cp1251') #считываем csv файл
    df_mean = [] #среднее значение лог доходностей за 2 года
    for year in range(2011, 2019, 2): #так как 2010 и 2020 года взяты не полностью
        condition = (df1['<DATE>'] >= year*10000) & (df1['<DATE>'] < (year+2)*10000) #условие для одного года
        df = pd.DataFrame()
        df['close'] = df1[condition]['<CLOSE>'] #отбираются цены закрытия для данного периода
        df['open'] = df1[condition]['<OPEN>'] #отбираются цены открытия для данного периода
        df['vol'] = df1[condition]['<VOL>'] #отбираются объем торгов в шт для данного периода
        df['rvol'] = ((df['close'] + df['open']) / 2) * df['vol']
        df = pd.DataFrame.diff(np.log(df['rvol'] ))[1::] #подсчет логдохоности
        df_mean.append(df.mean()) #добавление срднего значения за 2 года
    return round(st.shapiro(df_mean)[1], 5) #p-value за 2 года на всем периоде логдоходностей для одной компании
                                                                    
def p_v_all(file): #функция для вывода p-value логдоходностей компании за весь период для одной компании
    year_1 = 2011 #так как 2010 и 2020 года взяты не полностью
    year_2 = 2019
    df1 = pd.read_csv(file, sep=';', encoding='cp1251') #считываем csv файл
    condition = (df1['<DATE>'] >= year_1*10000) & (df1['<DATE>'] < year_2*10000) #условие для одного года                                                                
    df = pd.DataFrame()
    df['close'] = df1[condition]['<CLOSE>'] #отбираются цены закрытия для данного периода
    df['open'] = df1[condition]['<OPEN>'] #отбираются цены открытия для данного периода
    df['vol'] = df1[condition]['<VOL>'] #отбираются объем торгов в шт для данного периода
    df['rvol'] = ((df['close'] + df['open']) / 2) * df['vol']
    df = pd.DataFrame.diff(np.log(df['rvol'] ))[1::] #подсчет логдохоности
    return round(st.shapiro(df)[1], 5) #Возвращает p-value за весь период для одной компании

def check(p_t): #формирование таблицы, которая показывает принимается гипотеза или нет
    ch = []
    for p in p_t:
        if p >= 0.05:
            ch.append('+')
        else:
            ch.append('-')
    return ch

In [2]:
start = time.time()

tickers = ['AAPL', 'AXP', 'BA', 'CAT', 'IBM', 'INTC', 'KO', 'MSFT', 'XOM'] 
i = 0 #счётчик для индексов 
#списки p-value для малого, среднего и большого объема торгов
p_mal = list()
p_sr = list()
p_big = list()
#списки принятия основной гипотезы для малого, среднего и большого объема торгов
ch_mal = list()
ch_sr = list()
ch_big = list()

for ticker in tickers:
        with open(u'C:/Users/Андрюша/Desktop/Акции_курсовая/' + ticker + '.csv') as f:
            p_mal.append(p_v_kv(f))
        with open(u'C:/Users/Андрюша/Desktop/Акции_курсовая/' + ticker + '.csv') as f:
            p_sr.append(p_v_2_years(f)) 
        with open(u'C:/Users/Андрюша/Desktop/Акции_курсовая/' + ticker + '.csv') as f: 
            p_big.append(p_v_all(f))
        ch = check([p_mal[i], p_sr[i], p_big[i]])
        i += 1
        ch_mal.append(ch[0])
        ch_sr.append(ch[1])
        ch_big.append(ch[2])
        
print("Примерное время выполнения = {}".format(time.time() - start))

Примерное время выполнения = 12.07161808013916


In [3]:
P_znach_SHAPIR = pd.DataFrame(index = tickers)
P_znach_SHAPIR['Малый'] = p_mal #квартал 
P_znach_SHAPIR['Средний'] = p_sr #2 года
P_znach_SHAPIR['Большой'] = p_big #2011-2019 
P_znach_SHAPIR.to_excel(u'C:/Users/Андрюша/Desktop/Акции_Курсовая/P-значения_при_опр._уровне_объема_торгов.xlsx', index = True,  encoding = 'cp1251')
P_znach_SHAPIR

,Малый,Средний,Большой
AAPL,0.00035,0.05018,0.0
AXP,0.23434,0.96921,0.0
BA,0.01700,0.27855,0.0
CAT,0.00117,0.90719,0.0
IBM,0.02118,0.59729,0.0
INTC,0.98631,0.69809,0.0
KO,0.00013,0.21930,0.0
MSFT,0.28608,0.96516,0.0
XOM,0.00024,0.21443,0.0


In [4]:
check_gyp = pd.DataFrame(index = tickers)
check_gyp['Малый'] = ch_mal #квартал 
check_gyp['Средний'] = ch_sr #2 года
check_gyp['Большой'] = ch_big #2011-2019
check_gyp.to_excel(u'C:/Users/Андрюша/Desktop/Акции_Курсовая/Проверка_гипотезы_при_опр._уровне_объема_торгов.xlsx', index = True,  encoding = 'cp1251')
check_gyp

,Малый,Средний,Большой
AAPL,-,+,-
AXP,+,+,-
BA,-,+,-
CAT,-,+,-
IBM,-,+,-
INTC,+,+,-
KO,-,+,-
MSFT,+,+,-
XOM,-,+,-
